# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
# Import necessary libraries
import pandas as pd

# Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Display the first few rows of the dataframe
print(df.head())

# Get basic information about the dataframe
print(df.info())

# Check for missing values
print(df.isnull().sum())

# Investigate the 'id' column
# How many viewers also clicked?
viewers_clicked = df[df['action'] == 'click']['id'].nunique()
print("Number of viewers who clicked:", viewers_clicked)

# Are there any anomalies with the data; did anyone click who didn't view?
clicked_without_viewing = df[df['action'] == 'click']['id'].isin(df[df['action'] == 'view']['id']).sum()
print("Number of clicks without viewing:", clicked_without_viewing)

# Is there any overlap between the control and experiment groups?
overlap_count = df.groupby('id')['group'].nunique().eq(2).sum()
print("Number of users in both groups:", overlap_count)


                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None
timestamp    0
id           0
group        0
action       0
dtype: int64
Number of viewers who clicked: 1860
Number of clicks without viewing: 1860
Number of users in both groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [2]:
#Your code here
from statsmodels.stats.proportion import proportions_ztest

# Calculate the number of viewers and clickers for the control and experimental groups
control_viewers = df[(df['group'] == 'control') & (df['action'] == 'view')]['id'].nunique()
control_clickers = df[(df['group'] == 'control') & (df['action'] == 'click')]['id'].nunique()

experimental_viewers = df[(df['group'] == 'experiment') & (df['action'] == 'view')]['id'].nunique()
experimental_clickers = df[(df['group'] == 'experiment') & (df['action'] == 'click')]['id'].nunique()

# Calculate the observed proportion of viewers who clicked for control and experimental groups
control_proportion = control_clickers / control_viewers
experimental_proportion = experimental_clickers / experimental_viewers

# Perform two-sample proportion z-test
count = [control_clickers, experimental_clickers]
nobs = [control_viewers, experimental_viewers]
stat, pval = proportions_ztest(count, nobs, alternative='smaller')

# Print the results
print("Control group click-through rate:", control_proportion)
print("Experimental group click-through rate:", experimental_proportion)
print("p-value:", pval)

# Compare p-value to significance level
alpha = 0.05
if pval < alpha:
    print("Reject the null hypothesis: The experimental homepage is more effective than the control homepage.")
else:
    print("Fail to reject the null hypothesis: There is insufficient evidence to conclude that the experimental homepage is more effective than the control homepage.")


Control group click-through rate: 0.2797118847539016
Experimental group click-through rate: 0.3097463284379172
p-value: 0.004415037788297902
Reject the null hypothesis: The experimental homepage is more effective than the control homepage.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [4]:
#Your code here
import numpy as np
from scipy.stats import norm

# Step 1: Calculate the expected number of clicks for the experiment group
expected_clicks_experimental = experimental_viewers * control_proportion



# Print the results
print("Expected number of clicks for the experiment group (if same click-through rate as control):", expected_clicks_experimental)



Expected number of clicks for the experiment group (if same click-through rate as control): 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [5]:
#Your code here

variance = experimental_viewers * control_proportion * (1 - control_proportion)
std_dev = np.sqrt(variance)
z_score = (experimental_clickers - expected_clicks_experimental) / std_dev

print("Number of standard deviations from the estimate:", z_score)


Number of standard deviations from the estimate: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [7]:
#Your code here
p_value = norm.cdf(z_score)
print("P-value based on the normal distribution:", p_value)

P-value based on the normal distribution: 0.9998751347199305


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Yes, the results indicate that the experimental homepage is more effective than the control homepage. The p-value of approximately 0.004 is less than the commonly used significance level of 0.05, suggesting that the difference in click-through rates between the two groups is statistically significant.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.